## Sentinel-1 offset tracking

This application takes a pair of Sentinel-1 products and identifies and generates the coherence

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Test Site](#test-site)
* [Context](#context)
* [Applicability](#applicability)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [Strengths and Limitations](#strengths-limitations) 
* [License](#license)

As a data processor developer, I want to implement, and package an algorithm processing a pair of S1 SAR SLC datasets using the SNAP toolbox notebook archetype with the following processing steps:

A. S1 SAR processing per image:

* Application of orbit file (should wait for the orbit file a couple of days, since for coherence this is important)
* TOPS slice assembly (if necessary)
* TOPS split (if necessary)

B. For image pair:

* TOPS coregistration
* Coherence estimation (Given set of images from same orbit,  {t1, t2, t3, ... , tn}, two temporally adjacent images would * constitute an image pair, with the first one being the master. So the pairs would be: {t1 - t2, t2 - t3, ... , tn-1 - tn}.)
* TOPS deburst
* Multi-looking
* Terrain correction


TOPS --> Terrain Observation by Prograssive Scans

TOPS Slice assembly-->assemble subswaths back together

TOPS split -->split into subswaths



### <a name="service">Service definition

In [1]:
service = dict([('title', 'Sentinel-1 offset tracking'),
                ('abstract', 'Sentinel-1 offset tracking'),
                ('id', 'offset-tracking')])

In [2]:
max_velocity = dict([('id', 'max_velocity'),
               ('value', '8'),
               ('title', 'Max velocity'),
               ('abstract', 'Max velocity (m/day)')])

In [3]:
polarisation = dict([('id', 'polarisation'),
                     ('value', 'VV'),
                     ('title', 'Polarisation'),
                     ('abstract', 'Polarisation')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

These are the Sentinel-1 product identifiers

In [4]:
input_identifiers = ('S1A_IW_GRDH_1SDV_20181005T214239_20181005T214304_024006_029F71_16D6', 
                     'S1A_IW_GRDH_1SDV_20181005T214304_20181005T214335_024006_029F71_C8C4',
                     'S1A_IW_GRDH_1SDV_20180607T214243_20180607T214308_022256_026888_8C77',
                     'S1A_IW_GRDH_1SDV_20180607T214218_20180607T214243_022256_026888_C1E1')

**Input references**

These are the Sentinel-1 catalogue references

In [5]:
input_references = ('https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_GRDH_1SDV_20181005T214239_20181005T214304_024006_029F71_16D6', 
                    'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_GRDH_1SDV_20181005T214304_20181005T214335_024006_029F71_C8C4',
                    'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_GRDH_1SDV_20180607T214243_20180607T214308_022256_026888_8C77',
                    'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_GRDH_1SDV_20180607T214218_20180607T214243_022256_026888_C1E1')

**Data path**

This path defines where the data is staged-in. 

In [6]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [7]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
import ellip_snap_helpers
import xml.etree.ElementTree as ET
from scipy import interpolate
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import dateutil.parser as parser
import gc
import geopandas as gpd
from datetime import datetime

import matplotlib.pyplot as plt

import gzip
import shutil
import csv 
import gdal
import osr
import math 
import lxml.etree as etree

from shapely.wkt import loads
from shapely.geometry import mapping

import numpy as np

from shapely.geometry import box

import warnings
warnings.filterwarnings("ignore")

import glob

sys.path.append('/opt/anaconda/bin/')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors



import cioppy
ciop = cioppy.Cioppy()

## Read the products

### check if all the products have the same track number

In [9]:
product_TR = [None]*len(input_references)

for index,product_ref in enumerate(input_references):
    
    result_prod = ciop.search(end_point=product_ref,
                              params=[],
                              output_fields='startdate,track',
                              model='EOP')
    
    product_TR[index] = result_prod[0]['track']
    
    if index==0:
        
        slave_date = result_prod[0]['startdate'][:10]
    
    elif result_prod[0]['startdate'][:10] > slave_date:
    
        slave_date = result_prod[0]['startdate'][:10]

if not all(x == product_TR[0] for x in product_TR):

    raise ValueError('Not all products pertain the same track !')

In [10]:
slave_date

'2018-10-05'

#### Read the products

In [11]:
s1meta = "manifest.safe"

#slave_date = ''

slave_products = []
master_products = []

slave_prefix = []
master_prefix = []

dates = []

for index, identifier in enumerate(input_identifiers):

    
    s1_zip_file = os.path.join(data_path, identifier + '.zip') 
    s1_meta_file = os.path.join(data_path, identifier, identifier + '.SAFE', 'manifest.safe') 

    if os.path.isfile(s1_zip_file):
        s1prd = s1_zip_file
    elif os.path.isfile(s1_meta_file):
        s1prd = s1_meta_file

    #print identifier, s1prd
    reader = ProductIO.getProductReader("SENTINEL-1")
    product = reader.readProductNodes(s1prd, None)
    
    
    width = product.getSceneRasterWidth()
    height = product.getSceneRasterHeight()
    name = product.getName()
    start_date = parser.parse(product.getStartTime().toString()).isoformat()
    
    dates.append(start_date[:19])

    if start_date[:10] == slave_date:
        
            slave_products.append(s1prd)
            print("\nProduct: %s, %d x %d pixels of %s assigned as slave" % (name, width, height, start_date))
            slave_prefix.append(identifier.split('_')[-1]) 
            slave_data_take = identifier.split('_')[-2]
    else:
            master_products.append(s1prd)
            print("\nProduct: %s, %d x %d pixels of %s assigned as master" % (name, width, height, start_date))
            master_data_take = identifier.split('_')[-2]  
            master_prefix.append(identifier.split('_')[-1]) 

                        
output_name = 'S1_OFFSET_TRACKING_%s_%s_%s_%s_%s_%s_%s_%s' % (parser.parse(min(dates)).strftime('%Y%m%d%H%M%S'),
                                                     parser.parse(max(dates)).strftime('%Y%m%d%H%M%S'),
                                                     slave_data_take,
                                                     len(input_identifiers)/2, 
                                                     '_'.join(slave_prefix),
                                                     master_data_take,
                                                     len(input_identifiers)/2, 
                                                     '_'.join(master_prefix))

print("\nco-registered OUTPUT Img name is %s"%output_name)



Product: S1A_IW_GRDH_1SDV_20181005T214239_20181005T214304_024006_029F71_16D6, 25186 x 16835 pixels of 2018-10-05T21:42:39.420644 assigned as slave

Product: S1A_IW_GRDH_1SDV_20181005T214304_20181005T214335_024006_029F71_C8C4, 25193 x 20841 pixels of 2018-10-05T21:43:04.420939 assigned as slave

Product: S1A_IW_GRDH_1SDV_20180607T214243_20180607T214308_022256_026888_8C77, 25565 x 16828 pixels of 2018-06-07T21:42:43.361804 assigned as master

Product: S1A_IW_GRDH_1SDV_20180607T214218_20180607T214243_022256_026888_C1E1, 25558 x 16828 pixels of 2018-06-07T21:42:18.361866 assigned as master

co-registered OUTPUT Img name is S1_OFFSET_TRACKING_20180607214218_20181005214304_029F71_2_16D6_C8C4_026888_2_8C77_C1E1


In [12]:
mygraph = ellip_snap_helpers.GraphProcessor()

#### Read and if need assemble the products

In [13]:
operator = 'Read'

node_id = 'Read'

source_node_id = ''

In [14]:
if len(slave_products) > 1:
  
    slave_read_nodes = []
    
    # Read 
    for index, slave_identifier in enumerate(slave_products):
        
        operator = 'Read'
        
        parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
        node_id = 'Read-S(%s)' % index
        
        source_node_id = ''
        
        parameters['file'] = slave_identifier
                
        mygraph.add_node(node_id, operator, parameters, source_node_id)
    
        slave_read_nodes.append(node_id)
    
    
    source_nodes_id = slave_read_nodes
        
    operator = 'SliceAssembly'
       
    node_id = 'SliceAssembly-S'
    
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
    parameters['selectedPolarisations'] = polarisation['value']
    
    mygraph.add_node(node_id, operator, parameters, source_nodes_id)

    source_slave_orbit = node_id
    
else:
    
    operator = 'Read'
        
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
    node_id = 'Read-S'
        
    source_node_id = ''
        
    parameters['file'] = slave_products[0]
        
    mygraph.add_node(node_id, operator, parameters, source_node_id)
    
source_slave_orbit = node_id

In [15]:
if len(master_products) > 1:
  
    master_read_nodes = []
    
    # Read 
    for index, master_identifer in enumerate(master_products):
        
        operator = 'Read'
        
        parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
        node_id = 'Read-M(%s)' % index
        
        source_node_id = ''
        
        parameters['file'] = master_identifer
        
        mygraph.add_node(node_id, operator, parameters, source_node_id)
    
        master_read_nodes.append(node_id)
    
    
    source_nodes_id = master_read_nodes
        
    operator = 'SliceAssembly'
       
    node_id = 'SliceAssembly-M'
    
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
    parameters['selectedPolarisations'] = polarisation['value']
    
    mygraph.add_node(node_id, operator, parameters, source_nodes_id)

    source_master_orbit = node_id
    
else:
    
    operator = 'Read'
        
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
    node_id = 'Read-M'
        
    source_node_id = ''
        
    parameters['file'] = master_products[0]
        
    mygraph.add_node(node_id, operator, parameters, source_node_id)
    
source_master_orbit = node_id

In [16]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read-S(0)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_GRDH_1SDV_20181005T214239_20181005T214304_024006_029F71_16D6.zip</file>
    </parameters>
  </node>
  <node id="Read-S(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_GRDH_1SDV_20181005T214304_20181005T214335_024006_029F71_C8C4.zip</file>
    </parameters>
  </node>
  <node id="SliceAssembly-S">
    <operator>SliceAssembly</operator>
    <sources>
      <sourceProduct refid="Read-S(0)"/>
      <sourceProduct.1 refid="Read-S(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <selectedPolarisations>VV</selectedPolarisations>
    </parameters>
  </node>
  <node id="Read-M(0)">
    <operator>Read</operator>
    <so

### Apply orbit file

In [17]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File-S' 

source_node_id = source_slave_orbit

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

In [18]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [19]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File-M' 

source_node_id = source_master_orbit

In [20]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [21]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read-S(0)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_GRDH_1SDV_20181005T214239_20181005T214304_024006_029F71_16D6.zip</file>
    </parameters>
  </node>
  <node id="Read-S(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_GRDH_1SDV_20181005T214304_20181005T214335_024006_029F71_C8C4.zip</file>
    </parameters>
  </node>
  <node id="SliceAssembly-S">
    <operator>SliceAssembly</operator>
    <sources>
      <sourceProduct refid="Read-S(0)"/>
      <sourceProduct.1 refid="Read-S(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <selectedPolarisations>VV</selectedPolarisations>
    </parameters>
  </node>
  <node id="Read-M(0)">
    <operator>Read</operator>
    <so

### Land/sea mask

In [22]:
operator = 'Land-Sea-Mask'

node_id = 'Land-Sea-Mask-S' 

source_node_id = 'Apply-Orbit-File-S' 

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

parameters['landMask'] = 'false'

In [23]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [24]:
operator = 'Land-Sea-Mask'

node_id = 'Land-Sea-Mask-M' 

source_node_id = 'Apply-Orbit-File-M' 

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

parameters['landMask'] = 'false'

In [25]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

### DEM assisted coregistration

In [26]:
operator = 'DEM-Assisted-Coregistration'

node_id = 'DEM-Assisted-Coregistration' 

source_node_id = ['Land-Sea-Mask-S',
                  'Land-Sea-Mask-M']

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

In [27]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

### Subset

In [28]:
bbox = (119.5, -1.2, 120, -0.7)
#bbox = (119.5, -1.4, 120.05, -0.5)

In [29]:
operator = 'Subset'

node_id = 'Subset' 

source_node_id = 'DEM-Assisted-Coregistration'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

parameters['geoRegion'] = box(*bbox).wkt
parameters['copyMetadata'] = 'true'

In [30]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

### Sea Mask

In [31]:
#operator = 'Land-Sea-Mask'

#node_id = 'Land-Sea-Mask' 

#source_node_id = 'DEM-Assisted-Coregistration'

#parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
#parameters['landMask'] = 'false'
#parameters

In [32]:
#mygraph.add_node(node_id, operator, parameters, source_node_id)

### Offset tracking

In [42]:
operator = 'Offset-tracking'

node_id = 'Offset-tracking' 

#source_node_id = 'Land-Sea-Mask' 
source_node_id = 'Subset'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

In [43]:
parameters['maxVelocity'] = max_velocity['value']
parameters['fillHoles'] = 'false'
parameters['spatialAverage'] = 'false'
parameters['averageBoxSize'] = '5'

In [44]:
parameters

{'averageBoxSize': '5',
 'fillHoles': 'false',
 'gridAzimuthSpacing': '40',
 'gridRangeSpacing': '40',
 'maxVelocity': '8',
 'radius': '4',
 'registrationWindowHeight': '128',
 'registrationWindowWidth': '128',
 'resamplingType': 'BICUBIC_INTERPOLATION',
 'roiVector': None,
 'spatialAverage': 'false',
 'xCorrThreshold': '0.1'}

In [45]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [37]:
operator = 'Write'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
parameters['file'] = output_name
parameters['formatName'] = 'BEAM-DIMAP'

node_id = 'Write-Offset'

source_node_id = 'Offset-tracking'

mygraph.add_node(node_id, operator, parameters, source_node_id)

### Terrain correction

In [38]:
operator = 'Terrain-Correction'

source_node_id = 'Offset-tracking' 

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

node_id = 'Terrain-Correction'

mygraph.add_node(node_id, operator, parameters, source_node_id)

### Write

In [39]:
operator = 'Write'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
parameters['file'] = output_name
parameters['formatName'] = 'GeoTIFF-BigTIFF'

node_id = 'Write'

source_node_id = 'Terrain-Correction'

mygraph.add_node(node_id, operator, parameters, source_node_id)

In [40]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read-S(0)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_GRDH_1SDV_20181005T214239_20181005T214304_024006_029F71_16D6.zip</file>
    </parameters>
  </node>
  <node id="Read-S(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_GRDH_1SDV_20181005T214304_20181005T214335_024006_029F71_C8C4.zip</file>
    </parameters>
  </node>
  <node id="SliceAssembly-S">
    <operator>SliceAssembly</operator>
    <sources>
      <sourceProduct refid="Read-S(0)"/>
      <sourceProduct.1 refid="Read-S(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <selectedPolarisations>VV</selectedPolarisations>
    </parameters>
  </node>
  <node id="Read-M(0)">
    <operator>Read</operator>
    <so

In [46]:
mygraph.run()

Processing the graph
Process PID: 31004
Executing processing graph
Master: 05Oct2018
Slave: 05Oct2018 prep baseline: 0.0 temp baseline: 0.0
Slave: 07Jun2018 prep baseline: 16.406063 temp baseline: 120.000244

Master: 07Jun2018
Slave: 05Oct2018 prep baseline: -14.690111 temp baseline: -120.000244
Slave: 07Jun2018 prep baseline: 0.0 temp baseline: 0.0

....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.hsqldb.persist.Logger: dataFileCache open start
-- org.jblas INFO Deleting /tmp/jblas7795532952538609470/libjblas.so
-- org.jblas INFO Deleting /tmp/jblas7795532952538609470/libjblas_arch_flavor.so
-- org.jblas INFO Deleting /tmp/jblas7795532952538609470

Done.


*TODO*

- write output of dem coreg and look 
- find and convert velocity.csv to N/S displacement, (the file is in  output_name + '_offset' + .data
- create .properties or XML for all files 
- erase output_name + '_offset' .dim and output_name + '_offset' .data
- add legend

Then we'll

- ciop-release
- build 
- deploy


#### Get the Azimuth direction of the Satellite

In [161]:
#output_name = 'S1_OFFSET_TRACKING_20180607214218_20181005214304_029F71_2_16D6_C8C4_026888_2_8C77_C1E1'
xml = ET.parse(output_name + '.dim').getroot().find("Dataset_Sources/MDElem[@name='metadata']/MDElem[@name='Abstracted_Metadata']")
heading_ang = float(xml.find("MDATTR[@name='centre_heading']").text)

xml = ET.parse(output_name + '.dim').getroot().find("Dataset_Sources/MDElem[@name='metadata']/MDElem[@name='Abstracted_Metadata']")
height = int(xml.find("MDATTR[@name='num_output_lines']").text)
width = int(xml.find("MDATTR[@name='num_samples_per_line']").text)

#### Read Velocity.csv Results

In [162]:
lat=[]
lon=[]
range_shift=[]
azi_shift=[]
velocity=[]
with open(output_name + '.data' + '/vector_data/Velocity.csv','r') as csvfile:
    spamreader = csv.reader(csvfile,delimiter='\t')
    a = next(spamreader)
    a = next(spamreader)
    print a
    for row in spamreader:
        lon.append(row[3])
        lat.append(row[2]) 
        range_shift.append(row[9]) 
        azi_shift.append(row[10]) 
        velocity.append(row[7])

['Velocity', 'geometry:Point', 'mst_lat:Double', 'mst_lon:Double', 'slv_lat:Double', 'slv_lon:Double', 'distance:Double', 'velocity:Double', 'heading:Double', 'range_shift:Double', 'azimuth_shift:Double', 'style_css:String']


#### Read Velocity.tiff

In [163]:
ds = gdal.Open(output_name + '.tif')
band = ds.GetRasterBand(1)
vel_array = band.ReadAsArray()
gt = ds.GetGeoTransform()
proj = ds.GetProjection()
[height, width] = vel_array.shape

In [164]:
print vel_array.shape
print height

(7862, 8071)
7862


#### Build arrays : azimuth shift, range_shift and velocity

In [148]:
lat=np.array(lat,dtype=float)
lon=np.array(lon,dtype=float)
range_shift=np.array(range_shift,dtype=float)
azi_shift=np.array(azi_shift,dtype=float)
velocity=np.array(velocity,dtype=float)

#### Convert to N-S and E-W motion

In [149]:
lat_vec = np.linspace(gt[3], gt[3]+gt[5]*height, height)
lon_vec = np.linspace(gt[0], gt[0]+gt[1]*width, width)
print vel_array.shape
print lon_vec.shape
print width

(7862, 8071)
(8071,)
8071


In [150]:
ns_shift = azi_shift*math.cos(heading_ang) - range_shift*math.sin(heading_ang)
ew_shift = azi_shift*math.sin(heading_ang) + range_shift*math.cos(heading_ang)

xx, yy = np.meshgrid(lon_vec, lat_vec)
EW_shift = interpolate.griddata((lon, lat), ew_shift, (xx, yy))
NS_shift = interpolate.griddata((lon, lat), ns_shift, (xx, yy))

#### Mask out Sea

In [151]:
NS_shift_msk = np.where(vel_array != 0, NS_shift, 0)
EW_shift_msk = np.where(vel_array != 0, EW_shift, 0)
vel_array_msk = np.where(vel_array != 0, vel_array, 0)

#### Save as Geotiff

###### EW displacement:

In [55]:
driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create('SNAP_displacement_EW.tif', width, height, 1, gdal.GDT_Float64)
geotransform = gt
outdata.SetGeoTransform(geotransform)##sets same geotransform as input
srs = osr.SpatialReference()            # establish encoding
srs.ImportFromEPSG(4326)                # WGS84 lat/long
outdata.SetProjection(srs.ExportToWkt()) # export coords to file
outdata.GetRasterBand(1).WriteArray(EW_shift_msk)
outdata.GetRasterBand(1).SetNoDataValue(0)##if you want these values transparent
outdata.FlushCache() ##saves to disk!!

In [56]:
with open('SNAP_displacement_EW' + '.tif', 'rb') as f_in, gzip.open('SNAP_displacement_EW' + '.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)
    
#os.remove(output_name + '.tif')

###### NS displacement:

In [63]:
driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create('SNAP_displacement_NS.tif', width, height, 1, gdal.GDT_Float64)
geotransform = gt
outdata.SetGeoTransform(geotransform)##sets same geotransform as input
srs = osr.SpatialReference()            # establish encoding
srs.ImportFromEPSG(4326)                # WGS84 lat/long
outdata.SetProjection(srs.ExportToWkt()) # export coords to file
outdata.GetRasterBand(1).WriteArray(NS_shift_msk)
outdata.GetRasterBand(1).SetNoDataValue(0)##if you want these values transparent
outdata.FlushCache() ##saves to disk!!

In [58]:
with open('SNAP_displacement_NS' + '.tif', 'rb') as f_in, gzip.open('SNAP_displacement_NS' + '.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

###### Velocity:

In [113]:
driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create('SNAP_velocity.tif', width, height, 1, gdal.GDT_Float64)
geotransform = gt
outdata.SetGeoTransform(geotransform)##sets same geotransform as input
srs = osr.SpatialReference()            # establish encoding
srs.ImportFromEPSG(4326)                # WGS84 lat/long
outdata.SetProjection(srs.ExportToWkt()) # export coords to file
outdata.GetRasterBand(1).WriteArray(vel_array_msk)
outdata.GetRasterBand(1).SetNoDataValue(0)##if you want these values transparent
outdata.FlushCache() ##saves to disk!!

0

In [60]:
with open('SNAP_velocity' + '.tif', 'rb') as f_in, gzip.open('SNAP_velocity' + '.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

#### Results metadata

def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % sar)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition_parameters = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    acquisition = etree.SubElement(acquisition_parameters, '{%s}Acquisition' % sar)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)

    polarisation_channels = etree.SubElement(acquisition, '{%s}polarisationChannels' % eop)
    
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)

#### Get the result WKT


In [65]:
src = gdal.Open(output_name + '.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
        transform.TransformPoint(min_x, min_y)[1],
        transform.TransformPoint(max_x, max_y)[0],
        transform.TransformPoint(max_x, max_y)[1]).wkt

#### Create the EOP XML metadata file

In [66]:
import json
eop_metadata = dict()

eop_metadata['wkt'] = result_wkt
eop_metadata['startdate'] = parser.parse(min(dates)).strftime('%Y-%m-%dT%H:%M:%S')
eop_metadata['enddate'] = parser.parse(max(dates)).strftime('%Y-%m-%dT%H:%M:%S')

eop_xml = output_name + '.xml'
with open(eop_xml, 'wb') as file:
    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    file.write(json.dumps(eop_metadata))

#### Create the properties file for the reproducibility notebooks

for properties_file in ['result', 'stage-in']:

    if properties_file == 'result':
        title = 'Reproducibility notebook used for generating %s' % output_name
    else: 
        title = 'Reproducibility stage-in notebook for Sentinel-1 data for generating %s' % output_name
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (parser.parse(min(dates)).strftime('%Y-%m-%dT%H:%M:%S'), parser.parse(max(dates)).strftime('%Y-%m-%dT%H:%M:%S')))      
        file.write('geometry=%s' % (result_wkt))

In [103]:
master = []
slave = []
for items in input_identifiers:
    for im in master_products:
        if im.find(items) != -1:
            master.append(items)
    for im in slave_products:
        if im.find(items) != -1:
            slave.append(items)


In [104]:
for properties_file in ['SNAP_displacement_EW','SNAP_displacement_NS','SNAP_velocity']:
    if properties_file == 'SNAP_displacement_EW':
        title = 'SNAP Offset Tracking - EW displacement'
        units = 'meters'
    elif properties_file == 'SNAP_displacement_NS':
        title = 'SNAP Offset Tracking - NS displacement'
        units = 'meters'
    elif properties_file == 'SNAP_velocity':
        title = 'SNAP Offset Tracking - velocity' 
        units = 'meters/day'
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s - %s\n' % (parser.parse(min(dates)).strftime('%Y-%m-%dT%H:%M:%S'), parser.parse(max(dates)).strftime('%Y-%m-%dT%H:%M:%S')))      
        file.write('unit=%s\n' % units)
        file.write('master product(s)=%s\n' % master)
        file.write('slave product(s)=%s\n' % slave)

### Set color palette

### Compress the file

In [ ]:
#with open(output_name + '.tif', 'rb') as f_in, gzip.open(output_name + '.gz', 'wb') as f_out:
#    shutil.copyfileobj(f_in, f_out)
    
#os.remove(output_name + '.tif')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.